In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import skimage.io
import skimage.morphology

import os.path

import sys
sys.path.insert(0, '/home/jr0th/github/segmentation/code/')
import helper.metrics

import numpy as np

Using TensorFlow backend.


In [2]:
cp_label = '/home/jr0th/github/segmentation/experiments/BBBC022_hand/CP_labels/'
cp_label_tiff = '/home/jr0th/github/segmentation/experiments/BBBC022_hand/CP_labels_tiff/'
gt_label = '/home/jr0th/github/segmentation/experiments/BBBC022_hand/GT_labels/'

mat_cp_dir = '/home/jr0th/github/segmentation/experiments/BBBC022_hand/IoU_CP/'

In [3]:
# make CP labels png and relabel due to weird numbering
labels_cp_tiff = skimage.io.imread_collection(cp_label_tiff + "*.tiff")

# loop over all images
for index in range(len(labels_cp_tiff)):
    
    path = labels_cp_tiff.files[index]
    filename = os.path.basename(path)
    filename_wo_ext = os.path.splitext(filename)[0]
    
    label_cp_tiff = labels_cp_tiff[index]
    label_cp_png = skimage.morphology.label(label_cp_tiff)
    label_cp_png = skimage.img_as_ubyte(label_cp_png)
    
    skimage.io.imsave(cp_label + filename_wo_ext + '.png', label_cp_png)

/usr/local/lib/python3.5/site-packages/skimage/util/dtype.py:106: UserWarning: Possible sign loss when converting negative image of type int64 to positive image of type uint8.
  "%s to positive image of type %s." % (dtypeobj_in, dtypeobj))
/usr/local/lib/python3.5/site-packages/skimage/util/dtype.py:136: UserWarning: Downcasting int64 to uint8 without scaling because max value 81 fits in uint8
  warn(msg)
/usr/local/lib/python3.5/site-packages/skimage/util/dtype.py:136: UserWarning: Downcasting int64 to uint8 without scaling because max value 86 fits in uint8
  warn(msg)
/usr/local/lib/python3.5/site-packages/skimage/util/dtype.py:136: UserWarning: Downcasting int64 to uint8 without scaling because max value 66 fits in uint8
  warn(msg)
/usr/local/lib/python3.5/site-packages/skimage/util/dtype.py:136: UserWarning: Downcasting int64 to uint8 without scaling because max value 121 fits in uint8
  warn(msg)
/usr/local/lib/python3.5/site-packages/skimage/util/dtype.py:136: UserWarning: Down

In [ ]:
# buffer for results
results = np.empty(shape = (0, 4))

labels_cp = skimage.io.imread_collection(cp_label + "*.png")

# loop over all images
for index in range(len(labels_cp)):
    
    # print progress
    print(index + 1, '/', len(labels_cp))
    
    path = labels_cp.files[index]
    filename = os.path.basename(path)
    filename_wo_ext = os.path.splitext(filename)[0]
    
    # get labels
    label_cp = labels_cp[index]
    label_gt = skimage.io.imread(gt_label + filename_wo_ext + ".png")
    
    nb_gt_cells = np.max(label_gt)
    
    [nb_overdetection, nb_underdetection, mean_IoU, IoUs] = helper.metrics.compare_two_labels(label_cp, label_gt, True)
    
    np.save(mat_cp_dir + filename_wo_ext, IoUs)
    
    result = np.hstack((nb_overdetection, nb_underdetection, mean_IoU, nb_gt_cells))
    results = np.vstack((results, result))

print(results)

# calculate error ratio
sums = np.sum(results, axis = 0)
nb_gt_cells_total = sums[3]
nb_errors = sums[0] + sums[1]

err_ratio = nb_errors / nb_gt_cells_total
print(err_ratio)

np.savetxt('./results_table_CP.txt', results)
np.savetxt('./results_summary_CP.txt', np.array([nb_errors, nb_gt_cells_total, err_ratio]))